# Create Spotify playlist from list

In [79]:
import requests
import json
import re
import pandas as pd

title = "(?P<title>\w+)"
artist = "(?P<artist>\w+)"
holder = "(?P<word>\w+)"

# Generate track id list
def append_id(text):
    """extract and append track id to song_list
    """
    
    track = re.findall(r'"spotify:track:(.*?)"', text)[0]
    if track is not None:
        song_list.append(track)
    return song_list

def spotify_search(term):
    """search for songs on spotify using api and append_id to song_list
    """
    
    if isinstance(term, float):
        return False

    # api request to search
    url = "https://api.spotify.com/v1/search?q="+term+"&type=track"
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json', 'Authorization' : 'Bearer ' + token}
    r = requests.get(url, headers=headers)
    parsed_json = json.loads(r.text)
    
    if parsed_json['tracks']['total'] != 0:
        append_id(r.text)
    else:
        # song unavailable
        return False
    return True


# Generate spotify list
def spotify_add(term):
    """add song to spotify playlist by track id
    """
    # api request to add
    url = "https://api.spotify.com/v1/playlists/"+playlist_id+"/tracks?uris=spotify%3Atrack%3A"+term
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json', 'Authorization' : 'Bearer ' + token}
    r = requests.post(url, headers=headers)
    return


"""Text Processing"""

def define_pattern(pattern):
#     pattern = "%tracknumber% %artist% %%"

    var_names = re.findall(r'%(.*?)%', pattern)
    for var_name in var_names:
        pattern = re.sub("%" +var_name + "%", "(?P<" + var_name + r'>\\w+)', pattern)
        pattern = pattern.replace(r'(?P<>\w+)', r'(\w+)')

    print(pattern)
    return pattern


# Text processing
def remove_num(text):
    text = re.sub("(\d+)", "", text)
    return text

def remove_sym(text):
    text = re.sub(r'(\W)\1+', r' ', text)
    
    text = text.replace(".", " ").replace("，", "").replace("–", "").replace("*", "").replace(
        "-", " ").replace("·", " ").replace("’", " ").replace("/", " ")
#     text = text.replace("(", " ").replace("（", " ").replace(")", "").replace("）", "")
#     text = text.replace("<", " ").replace(">", "")
#     text = text.replace("《", " ").replace("》", " ")
#     text = text.replace("[", "").replace("]", "").replace("「", "").replace("」", "")
    return text

def remove_line(text):
    lines = [line.strip() for line in text.split('\n') if line.strip() != '']
    return lines
   
def split_line(line, pos1=0, pos2=-1):
    songs_to_search = line.split(" ")[pos1] + " " + line.split(" ")[pos2]
    return songs_to_search


def text_process(text, num=True):
    if not num:
        text = remove_num(text)
    text = remove_sym(text)
    
    text = text.replace("[粤]", "").replace("演唱：", "")
    text = text.replace("林隆旋", "林隆璇")
    text = text.replace("眞", "真")
    text = text.replace("\t", " ")
    #text = text.replace("  ", " ")
    lines = remove_line(text)
    return lines

def extract_text(lines, pattern):
    songs_to_search = lines
    
    for i in range(len(songs_to_search)):
        m = songs_to_search[i]
        
        try:
            m = re.match(pattern, m)
            songs_to_search[i] = ' '.join([m[k] for k in m.groupdict()])
        except:
            # the hardcoded way
            print("\n======hardcode======")
            print(songs_to_search[i])
            split_line(songs_to_search[i])
            print(songs_to_search[i])
    return songs_to_search

# Hard code to split lines
def split_text(lines, pos1=0, pos2=-1):
    songs_to_search = lines
    
    for i in range(len(songs_to_search)):
        songs_to_search[i] = split_line(songs_to_search[i], pos1, pos2)
            
    return songs_to_search


# Web data processing with pandas
def extract_web(link):
    website = pd.read_html(link)
    song_list = []

    all_songs = website[17]
    all_songs

    all_songs.rename(columns=cus_columns, inplace=True)

    if one:
        songs_to_search = all_songs[('十大中文金曲得獎名單（按頒獎次序排序）',  'title')]

    if two:
        all_songs['list1'] = all_songs["title1"] + ' ' +  all_songs["artist1"]
        all_songs['list2'] = all_songs["title2"] + ' ' +  all_songs["artist2"]

        songs_to_search = all_songs["list1"]
        songs_to_search = songs_to_search.append(all_songs["list2"])
        # songs_to_search.drop(index=25, inplace=True)
        
    return songs_to_search

# Extract multiple songs according to artists
def extract_composer(lines, artist_split="：", title_split="，"):
    songs_to_search = []
    
    for line in lines:
        artist = line.split(artist_split)[0][:3]
#         print(artist)
        title_str = line.split(artist_split)[1]

        if len(title_split) == 1:
            titles = title_str.split(title_split)
        else:
            titles = re.findall(title_between, artist_split + title_str)
#         print(titles)
            
        for title in titles:
            songs_to_search.append(artist + " " + title)
                
    return songs_to_search
# print(songs_to_search)

#### Request for API token.

https://developer.spotify.com/console/post-playlist-tracks/?playlist_id=&position=&uris=

In [29]:
# pattern = "(\w+) " +title+ "--" +artist+ ""
# string = text.splitlines()[0].replace("\t", " ")
string = "呢喃 黃鶯鶯)"
print("String: " + string)
# pattern = title+ " " +artist  # + " " +holder

m = re.match(pattern, string)
m.groupdict()
[m[k] for k in m.groupdict()]
string = ' '.join([m[k] for k in m.groupdict()])
print(string)
# print(m.groupdict().values())

String: 呢喃 黃鶯鶯)


AttributeError: 'NoneType' object has no attribute 'groupdict'

In [129]:
text = """1	–黄莺莺*	留不住的故事	
2	–蔡琴*	最后一夜	
3	–林忆莲*	爱上一个不回家的人	
4	–叶欢*	放我的真心在你的手心	
5	–苏芮*	亲爱的小孩	
6	–王芷蕾*	台北的天空	
7	–张雨生*	我是一颗秋天的树	
8	–王杰*	是否我真的一无所有	
9	–姜育恒*	痛快的歌	
10	–王芷蕾*	翩翩飞起	
11	–张雨生*	一天到晚游泳的鱼	
12	–姜育恒*	天天等天天问"""

playlist_id = '7DZV6oKBobOQVnYS8s89jD'
token = "BQAcg1bfiQotFYh4A6rKr6_I2vXPSCjqJGVoyrWpwWUdiMrewyGRTtWaaS-I-iunv-Ipa5B02jEfF94THYvAw3i-bgevgyJds5uxCyOvPcwQldmDjhNlpesjVR0tP_I-NA80Wn5cWzoe8grFZGw4tieRfMYqpSbWr6ubpX72IVD9HPZyKJS7avFBbvRHs9gYBSfS6Q"

# cus_columns = {"歌曲": "title", "主唱": "artist"}
# cus_columns = {"第一季金曲": "title1", "第二季金曲": "title2", "主唱": "artist1", "主唱.1": "artist2"}
link = None

pattern = "%title% %artist%"

if text:
    pattern = define_pattern(pattern)
    text = re.sub(r'《(.*?)》', r'', text)
    lines = text_process(text, num=False)
    
#     songs_to_search = lines
    songs_to_search = extract_text(lines, pattern)
#     songs_to_search = extract_composer(text, artist_split="——")
    songs_to_search = split_text(lines, 0, 1)
elif link:
    songs_to_search = web_process(link)

(?P<title>\w+) (?P<artist>\w+)


In [131]:
na = ""
song_list = []

for i in songs_to_search:
    if not spotify_search(i):
        print(" unavailable          ", end="")
        na = na + ", " + i
    print(i)
    
# create playlist and add
for i in song_list:
    spotify_add(i)
    
print("\n缺：" + na[2:])

黄莺莺 留不住的故事
蔡琴 最后一夜
林忆莲 爱上一个不回家的人
叶欢 放我的真心在你的手心
苏芮 亲爱的小孩
王芷蕾 台北的天空
 unavailable          张雨生 我是一颗秋天的树
王杰 是否我真的一无所有
姜育恒 痛快的歌
王芷蕾 翩翩飞起
张雨生 一天到晚游泳的鱼
姜育恒 天天等天天问

缺：张雨生 我是一颗秋天的树


### Custom 

In [119]:
text = """1987.8 黄莺莺《雪在烧》专辑  《你在我灵魂的边缘》 作曲：陈秀男
1988.9 姜育恒《跟往事干杯》专辑   制作：陈秀男《今夜星光灿烂》，《每一个辗转不眠的夜》作曲：陈秀男
1988.9 星星‧月亮‧太阳《我有自己的路走》专辑  制作：陈秀男《爱情电话》，《心愿》作曲：陈秀男
1988.10 叶欢《记得我们有约》专辑 制作：陈秀男《多说一些说甜一些》，《不眠的夜》作曲：陈秀男
"""

text = re.sub("《(.*?)》专辑", "", text)
text = remove_num(text)
text = remove_sym(text)

lines = remove_line(text)
lines = [line for line in lines if "《" in line]

songs_to_search = extract_composer(lines, artist_split="《", title_split=r"《(.*?)》")

In [116]:
text = """小虎队
《红蜻蜓》 专辑
《红蜻蜓》 词： [1]  李子恒 曲：长渕刚"""

lines = [line for line in text.split('\n') if '专辑' not in line]

songs_to_search = []
artist = ""

for line in lines:
    if "《" not in line:
        artist = line
    else:
        match = re.search(r'《(.*?)》', line)
        songs_to_search.append(artist + " " + match.group(1))

In [123]:
text = """一點點愉快，要用一生來感慨。（選自同恩「一個人遺憾」）

別問我生活累不累，靈魂的事，身體無法解決。（選自曾慶瑜「我背叛了愛」）

如果在你的懷裡甜蜜的死掉，拜託把憔悴的照片都燒掉，最美的一刻要你永遠記得我的好強。（選自徐懷鈺「如果在你的懷裡滿足的死掉」）

你放了太多愛，在我眼裡眉間，可以不用眺望天邊，也能喜悅。（選自蔡淳佳「眼裡眉間」）"""

text = remove_num(text)
text = remove_sym(text)

lines = remove_line(text)

songs_to_search = re.findall(r"（選自(.*?)）", text)